<a href="https://colab.research.google.com/github/Propa-Punam/Wifi-RSS-Crowdsensing/blob/main/temporary/told_to_be_exactly_like_boook_but_with_0_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

class CounterPropagationNetwork:
    def __init__(self, input_dim, num_zones):
        self.input_dim = input_dim  # n in the algorithm (number of APs)
        self.num_zones = num_zones  # m in the algorithm (number of clusters/zones)
        self.num_bits = int(np.ceil(np.log2(num_zones)))

        # Initialize weights for Kohonen layer (wij in the algorithm)
        self.kohonen_weights = np.zeros((num_zones, input_dim))

        # Initialize weights for Grossberg layer
        self.grossberg_weights = np.zeros((num_zones, self.num_bits))

        # Gray code mapping for zones
        self.gray_codes = {
            '203': [0, 0, 0],
            '204': [0, 0, 1],
            'l1': [0, 1, 1],
            'l2': [0, 1, 0],
            'l3': [1, 1, 0]
        }

        self.zone_mapping = {
            0: '203',
            1: '204',
            2: 'l1',
            3: 'l2',
            4: 'l3'
        }

    def preprocess_data(self, data):
        signal_columns = ['CSE-104', 'DataLab@BUET', 'Galaxy M124213', 'Hall of Fame',
                         'CSE-206', 'CSE-303', 'CSE-205', 'CSE-204', 'CSE-304', 'CSE-202']

        # Replace -100 with -130 as mentioned in the paper
        data[signal_columns] = data[signal_columns].replace(-100, -130)

        # Normalize RSSI values
        data[signal_columns] = (data[signal_columns] + 130) / 90

        return data[signal_columns].values

    def initialize_weights(self, training_data, rooms):
        # Step 1: Initialize weight vectors with sample mean vectors for each location
        for zone_idx, zone in enumerate(np.unique(rooms)):
            zone_data = training_data[rooms == zone]
            self.kohonen_weights[zone_idx] = np.mean(zone_data, axis=0)
            self.grossberg_weights[zone_idx] = self.gray_codes[zone]

    def train_kohonen(self, training_data, rooms, learning_rate=0.1, max_iterations=1000):
        # Following exactly Algorithm 3 from the paper

        # Step 1: Initialize weights with mean vectors
        self.initialize_weights(training_data, rooms)

        # Step 2: repeat
        for iteration in range(max_iterations):
            # Step 3: for each vector Vz,t = (rss1, rss2, ..., rssn)
            for idx, input_vector in enumerate(training_data):
                val_j = np.zeros(self.num_zones)

                # Step 4-5: for j = 1 to m do calculate VALj = Σi rssi × wij
                for j in range(self.num_zones):
                    val_j[j] = np.sum(input_vector * self.kohonen_weights[j])

                # Step 8: Assign input vector to cluster Lj for which VALj is maximum
                winner_idx = np.argmax(val_j)

                # Step 9-10: for i = 1 to n do wij(new) = wij(old) + β(rssj - wij(old))
                for i in range(self.input_dim):
                    old_weight = self.kohonen_weights[winner_idx][i]
                    self.kohonen_weights[winner_idx][i] = old_weight + learning_rate * (input_vector[i] - old_weight)

            # Decrease learning rate


    def predict(self, test_data):
        predictions = []

        for input_vector in test_data:
            # Calculate VALj for each cluster
            val_j = np.zeros(self.num_zones)
            for j in range(self.num_zones):
                val_j[j] = np.sum(input_vector * self.kohonen_weights[j])

            # Find winning neuron
            winner_idx = np.argmax(val_j)
            predicted_zone = self.zone_mapping[winner_idx]
            predictions.append(predicted_zone)

        return predictions

def evaluate_model(y_true, y_pred):
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    accuracy = correct / len(y_true)

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nConfusion Matrix:")
    for true_zone in sorted(set(y_true)):
        correct = sum(1 for true, pred in zip(y_true, y_pred)
                     if true == true_zone and true == pred)
        total = sum(1 for true in y_true if true == true_zone)
        print(f"Zone {true_zone}: {correct}/{total} ({correct/total*100:.2f}%)")

def main():
    # Load data
    train_df = pd.read_csv('/content/train.csv')
    test_df = pd.read_csv('/content/test.csv')

    # Initialize model
    cpn = CounterPropagationNetwork(input_dim=10, num_zones=5)

    # Preprocess data
    X_train = cpn.preprocess_data(train_df)
    y_train = train_df['room'].values

    # Train using Algorithm 3
    cpn.train_kohonen(X_train, y_train)

    # Test
    X_test = cpn.preprocess_data(test_df)
    y_test = test_df['room'].values
    predictions = cpn.predict(X_test)

    # Evaluate
    evaluate_model(y_test, predictions)

if __name__ == "__main__":
    main()

Accuracy: 0.00%

Confusion Matrix:
Zone 203: 0/12 (0.00%)
Zone 204: 0/11 (0.00%)
